In [1]:
import torch
import torchvision.models as models
import torch.nn as nn
from torchvision import datasets
import torchvision.transforms as transforms
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_transfer = models.resnet152(pretrained=True).to(device)
    
model_transfer.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 3)).to(device)
model_transfer.load_state_dict(torch.load('model_transfer_v32.pt', map_location=device))

FileNotFoundError: [Errno 2] No such file or directory: 'model_transfer_v32.pt'

In [ ]:
TEST_DATA_PATH = '../data/test'
transform = transforms.Compose([
                                transforms.Resize(224),
                                transforms.CenterCrop(224),
                                transforms.ToTensor()
                               ]) 

if not 'data_transfer' in locals():
    print("create empty data_transfer")
    data_transfer = {}
if not 'loaders_transfer' in locals():
    print("create empty loaders_transfer")
    loaders_transfer = {}
data_transfer['test'] = datasets.ImageFolder(TEST_DATA_PATH, transform=transform)
loaders_transfer['test'] = torch.utils.data.DataLoader(data_transfer['test'],
                                          batch_size=1,
                                          shuffle=True,
                                          num_workers=1)

import torch.optim as optim

criterion_transfer = nn.CrossEntropyLoss()

In [ ]:
import numpy as np

def test(loaders, model, criterion):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        data, target = data.to(device), target.to(device)
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

# call test function    
test(loaders_transfer, model_transfer, criterion_transfer)